In [60]:
import astropy.io.fits as fits
import numpy as np
import pylab as plt
from PIL import Image
from tensorflow import keras
from keras import utils
from keras.models import Model 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

In [5]:
hdulist = fits.open('train_data_01.fits')

In [55]:
flux = hdulist[0].data
labels = hdulist[1].data['label']
objid = hdulist[1].data['objid']
labels = utils.to_categorical(labels, 3)
labels = labels.astype(np.float32)
flux = flux.reshape(-1,3000,1).astype(np.float32)

train_idx, test_idx = train_test_split(np.arange(labels.shape[0]), test_size=0.1)
train_flux, train_labels, test_flux, test_labels = flux[train_idx], labels[train_idx], flux[test_idx], labels[test_idx]


In [49]:
model = Sequential()  
model.add(Conv1D(16, kernel_size=3, activation='relu', input_shape=(3000, 1)))  

model.add(Conv1D(32, kernel_size=3, activation='relu'))    
model.add(MaxPooling1D(pool_size=2))  
#model.add(Dropout(0.1))  
model.add(Flatten())  
model.add(Dense(64, activation='relu'))  
#model.add(Dropout(0.1))  
model.add(Dense(3, activation='softmax'))  
  
model.summary()

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_39 (Conv1D)              │ (None, 2998, 16)       │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_40 (Conv1D)              │ (None, 2996, 32)       │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_20 (MaxPooling1D) │ (None, 1498, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_19 (Flatten)            │ (None, 47936)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 64)             │     3,067,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,069,795 (11.71 MB)

 Trainable params: 3,069,795 (11.71 MB)

 Non-trainable params: 0 (0.00 B)

In [50]:
model.compile(
loss='categorical_crossentropy',
optimizer='Adam',
metrics=['accuracy'])
model.fit(train_flux,train_labels, epochs=5, 
validation_data=(test_flux,test_labels))

Epoch 1/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.6783 - loss: 64.1081 - val_accuracy: 0.8580 - val_loss: 2.3213
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.8874 - loss: 2.5862 - val_accuracy: 0.9290 - val_loss: 1.2033
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.8893 - loss: 5.4643 - val_accuracy: 0.9250 - val_loss: 1.8141
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9439 - loss: 1.1050 - val_accuracy: 0.9340 - val_loss: 0.4807
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9361 - loss: 0.9207 - val_accuracy: 0.9520 - val_loss: 1.4046


In [68]:
predicted_labels = model.predict(test_flux)
predicted_class = np.argmax(predicted_labels, axis=1)
test_class = np.argmax(test_labels, axis=1)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [77]:
test_objid = objid[test_idx].astype(np.float(32))

AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [72]:
catalog = {'objid':test_objid,'predicted_label':test_class}

In [76]:
catalog

{'objid': array([4895, 8761, 5714, 4115, 3855, 7367, 1781,  727, 2230, 3263, 5682,
        1294, 1097, 5509, 8451, 6070, 1457,  990, 4452, 7951, 6794, 6715,
         459, 3922,  685, 7084, 6813, 2487,  944, 2698, 6952, 9652, 3379,
        9705, 8460, 1305, 1408, 9545, 9688, 4665, 1607, 3122, 5974, 8212,
        6416, 8677, 4094, 6734, 4795, 9514, 7933, 9650,  555,  179, 1634,
        3590, 1207, 5594, 9367, 4268, 5344,  336, 1141, 8955, 2591, 5944,
        6043, 1730, 7770, 1695, 6103, 5708, 1105, 5345, 3747, 4560, 4464,
        1510, 3805, 4481, 6149, 6238, 3204, 7410, 5534, 2399,  833,  493,
        6492, 9758, 9116, 9154, 8132, 8402, 4042, 1505, 1058, 7884,  603,
        4675, 9497,  304, 8909, 9020, 6614, 1706, 8730, 3607, 4599, 5390,
        1081, 1148,  360, 9798, 7245, 2914, 8449, 3546, 5958,  973, 3309,
        7510, 1435, 7302, 5371, 2025, 4661, 8781, 8092,  509, 8369, 9753,
        5094, 4843, 6235, 4363, 7623, 9745, 8750, 9483, 7049, 4876, 7942,
        1747, 7787, 7363, 277

In [73]:
df = pd.DataFrame(catalog)